In [ ]:
#⚠️ Note: Outputs are cleared for GitHub compatibility.
All experiments and results are reproducible by running the notebook cells.


In [ ]:
# PaliGemma Vision-Language Model for Medical VQA

This notebook implements:
- Zero-shot inference using PaliGemma
- Evaluation on closed-ended and open-ended questions
- LoRA fine-tuning experiments
- Comparative analysis with CNN-GRU baseline
- Qualitative analysis of generated answers

Dataset: VQA-RAD


In [1]:
import torch
print("GPU available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")


GPU available: False
GPU name: No GPU


In [ ]:
#Mounted
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

base_path = "/content/drive/MyDrive/WOA7015_MedVQA"
os.makedirs(base_path, exist_ok=True)

print("Project directory:", base_path)


In [ ]:
import zipfile

zip_path = "/content/drive/MyDrive/VQA.zip"
extract_path = "/content/drive/MyDrive/WOA7015_MedVQA/data/"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("ZIP extracted successfully!")

In [ ]:
#Check dataset folder
!ls /content/drive/MyDrive/WOA7015_MedVQA/data/

In [ ]:
#Load & Inspect the Dataset
import os
import json

base_path = "/content/drive/MyDrive/WOA7015_MedVQA/data"

image_dir = os.path.join(base_path, "VQA_RAD Image Folder")
json_path = os.path.join(base_path, "VQA_RAD Dataset Public.json")

print("Image folder exists:", os.path.exists(image_dir))
print("JSON file exists:", os.path.exists(json_path))

In [ ]:
#Dataset Structure and Load the JSON File
with open(json_path, "r") as f:
    data = json.load(f)

print("Total QA samples:", len(data))
print("Sample entry:")
data[0]

In [ ]:
#Inspect One Sample
sample = data[0]

print("Image:", sample["image_name"])
print("Question:", sample["question"])
print("Answer:", sample["answer"])
print("Type:", sample["question_type"])


In [ ]:
#Question Type Distribution
def is_yes_no(ans):
    return isinstance(ans, str) and ans.lower() in ["yes", "no"]

yes_no = sum(is_yes_no(x["answer"]) for x in data)
open_ended = len(data) - yes_no

print("Yes/No questions:", yes_no)
print("Open-ended questions:", open_ended)

In [ ]:
#bar chart
import matplotlib.pyplot as plt
plt.figure()
plt.bar(["Yes/No", "Open-ended"], [yes_no, open_ended])
plt.title("Question Type Distribution")
plt.ylabel("Count")
plt.show()


In [ ]:
#Question Length
question_lengths = [len(x["question"].split()) for x in data]

print("Avg question length:", sum(question_lengths)/len(question_lengths))
print("Min question length:", min(question_lengths))
print("Max question length:", max(question_lengths))


In [ ]:
#Histogram
plt.figure()
plt.hist(question_lengths, bins=20)
plt.title("Question Length Distribution")
plt.xlabel("Number of words")
plt.ylabel("Frequency")
plt.show()

In [ ]:
#sample
sample = data[0]

print("Question:", sample["question"])
print("Answer:", sample["answer"])

from PIL import Image
import os
import matplotlib.pyplot as plt

img = Image.open(os.path.join(image_dir, sample["image_name"]))
plt.imshow(img)
plt.axis("off")
plt.show()

In [ ]:
#Created the Split
from sklearn.model_selection import train_test_split

# First split: Train (70%) and Temp (30%)
train_data, temp_data = train_test_split(
    data,
    test_size=0.30,
    random_state=42,
    shuffle=True
)

# Second split: Validation (15%) and Test (15%)
val_data, test_data = train_test_split(
    temp_data,
    test_size=0.50,
    random_state=42,
    shuffle=True
)

print("Train samples:", len(train_data))
print("Validation samples:", len(val_data))
print("Test samples:", len(test_data))

In [ ]:
#Import Libraries
import os
import re
import json
import torch
from torch.utils.data import Dataset
from PIL import Image
from torchvision import transforms
from collections import Counter


In [ ]:
#Define Image Transform
image_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [ ]:
#Check GPU & RAM
!nvidia-smi


In [ ]:
#Install Required Libraries
!pip install -q transformers accelerate sentencepiece bitsandbytes


In [ ]:
#Import Libraries
import torch
from transformers import AutoProcessor, AutoModelForVision2Seq
from PIL import Image

In [ ]:
# loggin my hugginface
from huggingface_hub import login
login()


In [ ]:
#Load PaliGemma
import torch
from transformers import AutoProcessor, AutoModelForVision2Seq

model_id = "google/paligemma-3b-mix-224"

processor = AutoProcessor.from_pretrained(model_id)

model = AutoModelForVision2Seq.from_pretrained(
    model_id,
    device_map="auto",
    load_in_8bit=True,
    torch_dtype=torch.float16
)

print("PaliGemma loaded successfully")


In [ ]:
#Prepare a Helper Function
def run_paligemma(image_path, question):#Function definition

    image = Image.open(image_path).convert("RGB") #Load and prepare the image

    prompt = f"Question: {question}\nAnswer:"  #Build the text prompt

    inputs = processor(
        images=image,
        text=prompt,             #Process image + text together
        return_tensors="pt"
    ).to(model.device)

    output = model.generate(
        **inputs,                #Generate the answer
        max_new_tokens=30,
        do_sample=False
    )

    answer = processor.decode(output[0], skip_special_tokens=True)  #Decode tokens back to text
    return answer


In [ ]:
#Test on a Closed-Ended (Yes/No) Question
sample = test_data[0]

image_path = os.path.join(image_dir, sample["image_name"])
question = sample["question"]
gt_answer = sample["answer"]

pred_answer = run_paligemma(image_path, question)

print("Question:", question)
print("Ground Truth:", gt_answer)
print("PaliGemma Answer:", pred_answer)


In [ ]:
# Find one open-ended question
for item in test_data:
    if isinstance(item["answer"], str) and item["answer"].lower() not in ["yes", "no"]:
        sample_oe = item
        break

image_path = os.path.join(image_dir, sample_oe["image_name"])
question = sample_oe["question"]
gt_answer = sample_oe["answer"]

pred_answer = run_paligemma(image_path, question)

print("Question:", question)
print("Ground Truth:", gt_answer)
print("PaliGemma Answer:", pred_answer)


In [ ]:
#Helper to clean answers
def normalize_yesno(text):
    text = text.lower()
    if "yes" in text:
        return "yes"
    if "no" in text:
        return "no"
    return "unknown"


In [ ]:
#Evaluate on Yes/No Test Set
correct = 0
total = 0
unknown = 0

for item in test_data:
    if isinstance(item["answer"], str) and item["answer"].lower() in ["yes", "no"]:
        image_path = os.path.join(image_dir, item["image_name"])
        question = item["question"]
        gt = item["answer"].lower()

        pred = run_paligemma(image_path, question)
        pred_norm = normalize_yesno(pred)

        if pred_norm == "unknown":
            unknown += 1
        elif pred_norm == gt:
            correct += 1

        total += 1

accuracy = correct / total
coverage = 1 - (unknown / total)

print("Approximate PaliGemma Yes/No Accuracy:", accuracy)
print("Answer Coverage (non-unknown):", coverage)


In [ ]:
open_ended_samples = []

for item in test_data:
    if isinstance(item["answer"], str) and item["answer"].lower() not in ["yes", "no"]:
        open_ended_samples.append(item)

print("Total open-ended questions:", len(open_ended_samples))


In [ ]:
for i in range(3):
    sample = open_ended_samples[i]

    image_path = os.path.join(image_dir, sample["image_name"])
    question = sample["question"]
    gt_answer = sample["answer"]

    pred_answer = run_paligemma(image_path, question)

    print(f"Example {i+1}")
    print("Question:", question)
    print("Ground Truth:", gt_answer)
    print("PaliGemma Answer:", pred_answer)
    print("-" * 50)


In [ ]:
def keyword_match(gt, pred):
    gt = gt.lower()
    pred = pred.lower()
    return gt in pred or pred in gt


In [ ]:
matches = 0
total = 0

for item in test_data:
    if isinstance(item["answer"], str) and item["answer"].lower() not in ["yes", "no"]:

        image_path = os.path.join(image_dir, item["image_name"])
        question = item["question"]

        pred = run_paligemma(image_path, question)

        if keyword_match(item["answer"], pred):
            matches += 1

        total += 1

score = matches / total
print("Approximate keyword match rate (open-ended):", score)


In [ ]:
count = 0

for item in test_data:
    if (
        isinstance(item["answer"], str)
        and item["answer"].lower() in ["yes", "no"]
        and count < 3
    ):
        pred = run_paligemma(
            os.path.join(image_dir, item["image_name"]),
            item["question"]
        )

        print("Q:", item["question"])
        print("GT:", item["answer"])
        print("Pred:", pred)
        print("-" * 40)

        count += 1


In [ ]:
#Install Required Libraries
!pip install -q peft datasets accelerate


In [ ]:
#Prepare Training Samples
def format_sample(sample):
    prompt = f"Question: {sample['question']}\nAnswer:"
    answer = sample["answer"].lower()
    return {
        "image_path": os.path.join(image_dir, sample["image_name"]),
        "prompt": prompt,
        "answer": answer
    }

train_lora = [format_sample(x) for x in train_data if str(x["answer"]).lower() in ["yes", "no"]]
val_lora   = [format_sample(x) for x in val_data if str(x["answer"]).lower() in ["yes", "no"]]

print("LoRA train samples:", len(train_lora))

In [ ]:
#Prepare Training Samples
def format_sample(sample):
    prompt = f"Question: {sample['question']}\nAnswer:"
    answer = str(sample["answer"]).lower()  # ensure string

    return {
        "image_path": os.path.join(image_dir, sample["image_name"]),
        "prompt": prompt,
        "answer": answer
    }


train_lora = [format_sample(x) for x in train_data]
val_lora   = [format_sample(x) for x in val_data]


print("LoRA train samples:", len(train_lora))

In [ ]:
#Create Dataset Class
from torch.utils.data import Dataset
import torch

class PaliGemmaLoRADataset(Dataset):
    def __init__(self, samples, processor, max_length=256):
        self.samples = samples
        self.processor = processor
        self.max_length = max_length

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        item = self.samples[idx]
        image = Image.open(item["image_path"]).convert("RGB")

        # Full text = prompt + answer
        full_text = item["prompt"] + " " + item["answer"]

        encoding = self.processor(
            images=image,
            text=full_text,
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=self.max_length
        )

        input_ids = encoding["input_ids"].squeeze(0)
        labels = input_ids.clone()

        # Mask prompt tokens
        prompt_len = len(
            self.processor.tokenizer(
                item["prompt"],
                truncation=True,
                max_length=self.max_length
            ).input_ids
        )

        labels[:prompt_len] = -100  # ignore prompt tokens in loss

        encoding = {k: v.squeeze(0) for k, v in encoding.items()}
        encoding["labels"] = labels

        return encoding


In [ ]:
#Apply LoRA to PaliGemma
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj"],
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

In [ ]:
#Training Arguments
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./paligemma_lora",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=2,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=50,
    save_strategy="no",
    report_to="none"
)


In [ ]:
train_dataset = PaliGemmaLoRADataset(train_lora, processor)

In [ ]:
#Trainer Setup & Training
from transformers import Trainer

train_dataset = PaliGemmaLoRADataset(train_lora, processor)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)

trainer.train()


In [ ]:
#testing sample
sample = test_data[0]
pred = run_paligemma(
    os.path.join(image_dir, sample["image_name"]),
    sample["question"]
)

print("Question:", sample["question"])
print("Ground Truth:", sample["answer"])
print("After LoRA:", pred)


In [ ]:
#Plot the LoRA Loss Graph
import matplotlib.pyplot as plt

steps = [50,100,150,200,250,300,350,400,450,500,550,600,650,700,750]
losses = [
    12.8675, 12.4981, 12.4836, 12.4780, 12.4748,
    12.4727, 12.4711, 12.4699, 12.4690, 12.4683,
    12.4677, 12.4673, 12.4670, 12.4668, 12.4666
]

plt.figure()
plt.plot(steps, losses, marker="o")
plt.xlabel("Training Steps")
plt.ylabel("Training Loss")
plt.title("LoRA Fine-Tuning Loss Curve (PaliGemma)")
plt.grid(True)
plt.show()


In [ ]:
def run_paligemma_closed(image_path, question):
    image = Image.open(image_path).convert("RGB")

    prompt = (
        "<image>\n"
        "You are a medical assistant.\n"
        "Answer using only one word: Yes or No.\n"
        f"Question: {question}\n"
        "Answer:"
    )

    inputs = processor(
        images=image,
        text=prompt,
        return_tensors="pt"
    ).to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=3,
        do_sample=False
    )

    decoded = processor.decode(outputs[0], skip_special_tokens=True).lower()

    if "yes" in decoded:
        return "yes"
    if "no" in decoded:
        return "no"
    return "unknown"


In [ ]:
correct = 0
total = 0

for item in test_data:
    if isinstance(item["answer"], str) and item["answer"].lower() in ["yes", "no"]:
        image_path = os.path.join(image_dir, item["image_name"])
        question = item["question"]
        gt = item["answer"].lower()

        pred = run_paligemma_closed(image_path, question)

        if pred == gt:
            correct += 1
        total += 1

accuracy = correct / total if total > 0 else 0
print("Approximate PaliGemma + LoRA Yes/No Accuracy:", accuracy)


In [ ]:
open_ended_samples = []

for item in test_data:
    if isinstance(item["answer"], str) and item["answer"].lower() not in ["yes", "no"]:
        open_ended_samples.append(item)

print("Total open-ended test samples:", len(open_ended_samples))


In [ ]:
num_examples = 3  # you can set 3–5

for i in range(num_examples):
    sample = open_ended_samples[i]

    image_path = os.path.join(image_dir, sample["image_name"])
    question = sample["question"]
    gt_answer = sample["answer"]

    pred_answer = run_paligemma(image_path, question)

    print(f"Example {i+1}")
    print("Question:", question)
    print("Ground Truth:", gt_answer)
    print("Model Answer:", pred_answer)
    print("-" * 60)


In [ ]:
#Closed-Ended Accuracy Comparison
import matplotlib.pyplot as plt

models = [
    "CNN-GRU\n(Trained)",
    "PaliGemma\n(Zero-shot)",
    "PaliGemma + LoRA"
]

accuracies = [
    0.71,   # CNN-GRU
    0.59,   # PaliGemma zero-shot
    0.49    # PaliGemma + LoRA (final)
]

plt.figure(figsize=(7,5))
plt.bar(models, accuracies)
plt.ylabel("Yes/No Accuracy")
plt.ylim(0, 1)
plt.title("Closed-Ended (Yes/No) Accuracy Comparison")
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()


In [ ]:
#Open-Ended Ability
models = ["CNN-GRU", "PaliGemma", "PaliGemma + LoRA"]
open_ended_scores = [0.0, 0.21, 0.30]  # approx keyword match / qualitative

plt.figure(figsize=(7,5))
plt.bar(models, open_ended_scores)
plt.ylabel("Approx. Open-Ended Match Rate")
plt.ylim(0, 1)
plt.title("Open-Ended Question Performance Comparison")
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()
